$\newcommand{\To}{\Rightarrow}$

In [1]:
import os, sys
sys.path.append(os.path.split(os.getcwd())[0])

In [2]:
from kernel.type import TFun, boolT
from kernel.term import Term, Var, Const, Abs, Bound
from data import nat
from data.nat import natT, zero, plus, times
from logic import logic

## Lambda calculus

In the previous section, we discussed how to construct terms consisting of variables, constants, and function application. The relevant constructors are `Var`, `Const`, and `Comb`. In this section, we discuss two more constructors: `Abs` and `Bound`, that complete the picture for representation of terms in *lambda calculus*.

The motivation is as follows: we have already noted that terms can have function type. For example, in the previous section, we can declare a variable $f$ of type $nat \To nat$ by `Var("f", TFun(natT, natT))`. We have also encountered constants that have function type, for example the addition operator. However, we have not said anything about how to construct new examples of such functions.

In principle, any well-defined rule for computing the output from the input should be representable as a function. For example, there should be a function that takes as input a natural number $n$, and outputs $n+2$. In higher-order logic (also known as *simply-typed lambda calculus*), we can represent such functions as *lambda terms*. The above function can be written (in mathematical notation) as:

$$ \lambda n. n + 2 $$

Here $n$ (the variable right after $\lambda$) is known as a *bound variable*, in the sense that it is associated to the $\lambda$ sign directly in front of it, and is valid only in the scope of that $\lambda$ sign. It is important to note that *the name of the bound variable does not matter*. The expression $\lambda n. n + 2$ means the same thing as the expression $\lambda m. m + 2$. Both represent functions that add 2 to its input. We say that two terms are *$\alpha$-equivalent* if one can be changed to the other by changing the names of some bound variables.

When representing terms in higher-order logic, we would like to be able to quickly tell whether two terms are $\alpha$-equivalent. This motivates the use of *de Bruijn index* (named after Dutch mathematician Nicolaas Govert de Bruijn). Following this method, the bound variables are (in principle) unnamed, and whenever one needs to refer to a bound variable, one uses a sign $B_i$ where $i$ counts the depth of the location of reference with respect to the lambda sign of that variable. We follow the convention that the counting begins at 0. For example, the above function is represented using de Bruijn index as:

$$ \lambda\_. B_0 + 2 $$

Here we use an underscore to denote a bound variable that is unnamed. Another example: the expression $\lambda x. \lambda y. x + y$ is represented as $\lambda\_. \lambda\_. B_1 + B_0$ using de Bruijn indices. This is because the location where $x$ occurs is separated from the $\lambda$ sign that bounds it (the first $\lambda$ sign) by one $\lambda$ sign in the middle, while the location where $y$ occurs is directly after the $\lambda$ sign that bounds it (the second $\lambda$ sign).

In Python, we can construct a lambda expression as follows:

In [3]:
two = nat.to_binary(2)
f = Abs("x", natT, plus(Bound(0), two))
print(f)

%x. plus x (bit0 (Suc zero))


Here, `Abs` is the constructor for a lambda term. The first argument is the *suggested* name of the bound variable. It is used for printing only (and perhaps as a starting point when names of new variables need to be invented during proof). As we shall see, it does not matter when comparing terms. The second argument is the type of the bound variable, which *is* significant (different types of bound variables give different terms). The third argument is the body of the lambda term. In the body, bound variables are refered to by `Bound(n)`, where $n$ is a natural number.

The default printing function rather faithfully represents the structure of the term. Note `Bound(0)` is printed as $x$. However, the result is still difficult to read. In particular, the plus operator is not printed in infix form, and the number 2 is printed in a rather strange way (the representation of natural numbers, as well as the `to_binary` function used above, will be discussed in a later section). Therefore, from now on we switch to the more sophisticated printing function.

In [4]:
from syntax import printer
from logic import basic
thy = basic.load_theory('nat')

In [5]:
print(printer.print_term(thy, f, unicode=True))

λx. x + 2


The following expression stands for a function that takes two natural numbers $x$ and $y$ as input, and returns $x+2y$.

In [6]:
f2 = Abs("x", natT, Abs("y", natT, plus(Bound(1), times(two, Bound(0)))))
print(printer.print_term(thy, f2, unicode=True))

λx. λy. x + 2 * y


The constructor `Abs` actually can take any odd number of arguments. The above code can be simplified as follows:

In [7]:
f2 = Abs("x", natT, "y", natT, plus(Bound(1), times(two, Bound(0))))
print(printer.print_term(thy, f2, unicode=True))

λx. λy. x + 2 * y


The types of $f$ and $f_2$ are as expected (recall `checked_get_type` will perform type-checking on the term, in addition to returning the type of the term).

In [8]:
print(f.checked_get_type())
print(f2.checked_get_type())

nat => nat
nat => nat => nat


`Abs` can also be used to construct predicates or binary relations.

In [9]:
eq_nat = Const("equals", TFun(natT, natT, boolT))
P = Abs("x", natT, logic.disj(eq_nat(Bound(0), zero), eq_nat(Bound(0), two)))
print(printer.print_term(thy, P, unicode=True))
print(P.checked_get_type())

R = Abs("x", natT, "y", natT, eq_nat(Bound(1), plus(Bound(0), two)))
print(printer.print_term(thy, R, unicode=True))
print(R.checked_get_type())

λx. x = 0 ∨ x = 2
nat => bool
λx. λy. x = y + 2
nat => nat => bool


Here we encounter a minor problem: the method `Term.mk_equals` cannot be used to construct the equality in this case, because it needs to compute the types of both of its arguments (to instantiate the type of the equality constant). However, it is impossible to compute the type of `Bound(0)` independent of its context. Hence, we construct the equality constant separately. In general, it is not very convenient to construct lambda terms using `Abs`. Later we will see another way that is often more convenient.

## $\beta$-conversion

In the previous section, we constructed lambda terms using the `Abs` constructor. These are supposed to represent functions. What happens when we apply such functions an argument? Well, initially nothing happens:

In [10]:
print(printer.print_term(thy, f, unicode=True))
t = f(two)
print(printer.print_term(thy, t, unicode=True))

λx. x + 2
(λx. x + 2) 2


The `Comb` constructor (invoked through the `__call__` method of $f$) simply combines its two arguments, performing no function evaluation. To actually evaluate a function application, we need to use the `beta_conv` method, so named because function evaluation in lambda calculus is called *$\beta$-conversion*.

In [11]:
t2 = t.beta_conv()
print(printer.print_term(thy, t2))

2 + 2


Now, the argument 2 is substituted into the function. The addition $2+2$ is still not evaluated: the general rule is that no evaluation is performed unless explicitly called for. We will discuss evaluation of arithmetic on natural numbers in a later section.

Let's see a more complicated example:

In [12]:
three = nat.to_binary(3)
t3 = f2(two, three)
print(printer.print_term(thy, t3, unicode=True))

t4 = t3.beta_conv()   # raises TermSubstitutionException

(λx. λy. x + 2 * y) 2 3


TermSubstitutionException: 

Oops... Here `beta_conv` failed because the function part of $t_3$ is not a lambda term: it is a lambda term applied to 2. To fully evaluate $f_2$ on two arguments 2 and 3, we need to apply them one at a time, performing $\beta$-conversion:

In [13]:
t3 = f2(two).beta_conv()
print(printer.print_term(thy, t3, unicode=True))

t4 = t3(three).beta_conv()
print(printer.print_term(thy, t4, unicode=True))

λy. 2 + 2 * y
2 + 2 * 3


A more convenient method is `beta_norm`, which performs all $\beta$-conversions on subterms:

In [14]:
t5 = f2(two, three).beta_norm()
print(printer.print_term(thy, t5, unicode=True))

2 + 2 * 3


## Abstraction

The dual of $\beta$-conversion is *abstraction*. It takes a term containing a variable $x$, and creates a lambda term where $x$ is replaced by a bound variable. For example:

In [15]:
x = Var("x", natT)
t = plus(x, two)
t2 = Term.mk_abs(x, t)
print(printer.print_term(thy, t2, unicode=True))

λx. x + 2


The `mk_abs` function is often more convenient for constructing abstractions. At the expense of defining extra variables, it removes the need to think about index of bound variables. In addition, types can be computed for all terms in the body, making it possible to use functions such as `mk_equals`. We give two examples from above.

In [16]:
x = Var("x", natT)
y = Var("y", natT)

P = Term.mk_abs(x, Term.mk_abs(y, logic.disj(Term.mk_equals(x, zero), eq_nat(x, two))))
print(printer.print_term(thy, P, unicode=True))

R = Term.mk_abs(x, Term.mk_abs(y, Term.mk_equals(x, plus(y, two))))
print(printer.print_term(thy, R, unicode=True))

λx. λy. x = 0 ∨ x = 2
λx. λy. x = y + 2


## Quantifiers in predicate logic

Predicate logic extends propositional logic by adding two quantifiers: forall ($\forall$) and exists ($\exists$). In higher-order logic, both operators are represented as constants of type $('a \To bool) \To bool$. This can be explained as follows, taking the forall quantifier as an example. A forall expression in mathematics has the form

$$ \forall x. P(x) $$

Here $x$ is a bound variable. In (untyped) first-order logic, there are only two types of terms: objects and propositions, and $x$ can only range over objects. The main distinction between higher-order and first-order logic is that in higher-order logic, the bound variable of quantifiers can be of any type, including function types. Hence, we designate the type of the bound variable by the type variable $'a$. Then, the predicate $P$ has type $'a \To bool$. Any forall expression is a function taking a predicate $P$ of type $'a \To bool$ as input, and outputs a boolean value (whether $P$ is true on all of $'a$). Hence, its type must be $('a \To bool) \To bool$.

Forall and exists expressions are constructed as follows.

In [17]:
P = Var("P", TFun(natT, boolT))
x = Var("x", natT)
t1 = Term.mk_all(x, P(x))
print(repr(t1))
print(t1)

Comb(Const(all,(nat => bool) => bool),Abs(x,nat,Comb(Var(P,nat => bool),Bound 0)))
all (%x. P x)


As indicated by the output of the default printer, $\forall x. P(x)$ is represented as the constant `all` applied to the abstraction $\lambda x. P(x)$. Check the `repr` of $t_1$ carefully to make sure you understand everything. The more sophisticated printer outputs $t_1$ as follows:

In [18]:
print(printer.print_term(thy, t1, unicode=True))

∀x. P x


An exists quantifier is constructed in an analogous way, this time using a function in the `logic` module.

In [19]:
t2 = logic.mk_exists(x, P(x))
print(repr(t2))
print(t2)
print(printer.print_term(thy, t2, unicode=True))

Comb(Const(exists,(nat => bool) => bool),Abs(x,nat,Comb(Var(P,nat => bool),Bound 0)))
exists (%x. P x)
∃x. P x


The type of $t_1$ and $t_2$ are booleans, as expected.

In [20]:
print(t1.checked_get_type())
print(t2.checked_get_type())

bool
bool
